In [1]:
import sys

import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from Tests_ODaDiL import test_dadil, test_odadil, test_forgetting_odadil

C:\Users\SL276123\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ot\backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())
C:\Users\SL276123\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
list_of_datasets = []
for i in range(1, 11):
    dataset = np.load(f'data/toy_non_linear_100d_dataset_{i}.npy')
    list_of_datasets.append(dataset)

In [3]:
n_samples = 1000
batch_size = 200
n_atoms = 3
n_classes = 10

In [4]:
results = test_dadil(list_of_datasets, n_samples, n_classes, n_atoms, batch_size)

0/10
It 1/100, Loss: 6209250.699999999
It 2/100, Loss: 4204245.8
It 3/100, Loss: 2838540.95
It 4/100, Loss: 2083650.4500000002
It 5/100, Loss: 1786215.275
It 6/100, Loss: 1650921.35
It 7/100, Loss: 1597067.2249999999
It 8/100, Loss: 1543512.9499999997
It 9/100, Loss: 1522515.95
It 10/100, Loss: 1504265.3000000003
It 11/100, Loss: 1498196.625
It 12/100, Loss: 1490053.5500000003
It 13/100, Loss: 1502017.125
It 14/100, Loss: 1462396.375
It 15/100, Loss: 1438490.8499999999
It 16/100, Loss: 1458895.35
It 17/100, Loss: 1430328.225
It 18/100, Loss: 1415622.85
It 19/100, Loss: 1434415.6749999998
It 20/100, Loss: 1422246.575
It 21/100, Loss: 1402008.8000000003
It 22/100, Loss: 1415507.8
It 23/100, Loss: 1386256.9500000002
It 24/100, Loss: 1374237.75
It 25/100, Loss: 1370744.45
It 26/100, Loss: 1361782.025
It 27/100, Loss: 1348585.225
It 28/100, Loss: 1367515.1999999997
It 29/100, Loss: 1338852.4
It 30/100, Loss: 1350442.875
It 31/100, Loss: 1332413.425
It 32/100, Loss: 1324365.7750000001
It 33/

In [5]:
results

{'lin': {'wda': 0.6477999999999999,
  'e': 0.5986,
  'e_ot': 0.6491799999999999,
  'r': 0.6237,
  'r_ot': 0.5994999999999999},
 'rbf': {'wda': 0.6413,
  'e': 0.46840000000000004,
  'e_ot': 0.6289300000000001,
  'r': 0.4699000000000001,
  'r_ot': 0.6618000000000002},
 'RF': {'wda': 0.6448,
  'e': 0.6533,
  'e_ot': 0.5786400000000002,
  'r': 0.6512,
  'r_ot': 0.5767100000000001}}

In [4]:
results, before_online_results, after_online_results = test_forgetting_odadil(list_of_datasets, n_samples, n_classes, n_atoms, batch_size)

0/10
It 1/100, Loss: 6255497.8
It 2/100, Loss: 4069844.85
It 3/100, Loss: 2699118.5500000003
It 4/100, Loss: 1985062.7000000002
It 5/100, Loss: 1683819.325
It 6/100, Loss: 1529192.6749999998
It 7/100, Loss: 1472963.85
It 8/100, Loss: 1428292.6
It 9/100, Loss: 1416912.5999999996
It 10/100, Loss: 1382569.6249999998
It 11/100, Loss: 1363275.275
It 12/100, Loss: 1361846.1249999998
It 13/100, Loss: 1354688.3
It 14/100, Loss: 1343397.375
It 15/100, Loss: 1346067.35
It 16/100, Loss: 1323858.925
It 17/100, Loss: 1318959.325
It 18/100, Loss: 1303867.7000000002
It 19/100, Loss: 1299157.2
It 20/100, Loss: 1298288.15
It 21/100, Loss: 1284577.8250000002
It 22/100, Loss: 1273357.375
It 23/100, Loss: 1277433.0
It 24/100, Loss: 1273781.525
It 25/100, Loss: 1262150.2
It 26/100, Loss: 1246810.3
It 27/100, Loss: 1258841.125
It 28/100, Loss: 1236156.775
It 29/100, Loss: 1231542.925
It 30/100, Loss: 1230996.425
It 31/100, Loss: 1221885.65
It 32/100, Loss: 1202079.35
It 33/100, Loss: 1207510.2
It 34/100, Lo

In [5]:
results

{'lin': {'wda': 0.6477999999999999,
  'e': 0.7225,
  'e_ot': 0.6306799999999999,
  'r': 0.7086,
  'r_ot': 0.5772999999999999},
 'rbf': {'wda': 0.6413,
  'e': 0.4452999999999999,
  'e_ot': 0.59463,
  'r': 0.42890000000000006,
  'r_ot': 0.6539999999999999},
 'RF': {'wda': 0.6634,
  'e': 0.6845,
  'e_ot': 0.56812,
  'r': 0.6648,
  'r_ot': 0.5654999999999999}}

In [6]:
before_online_results

{'lin': {'r': [0.766, 0.766, 0.766, 0.766, 0.766],
  'r_ot': [0.5730000000000001,
   0.5730000000000001,
   0.5730000000000001,
   0.5730000000000001,
   0.5730000000000001]},
 'rbf': {'r': [0.393, 0.393, 0.393, 0.393, 0.393],
  'r_ot': [0.7539999999999999,
   0.7539999999999999,
   0.7539999999999999,
   0.7539999999999999,
   0.7539999999999999]},
 'RF': {'r': [0.791, 0.778, 0.795, 0.787, 0.788],
  'r_ot': [0.5579999999999999,
   0.5579999999999999,
   0.5579999999999999,
   0.5579999999999999,
   0.5579999999999999]}}

In [7]:
after_online_results

{'lin': {'r': [], 'r_ot': []},
 'rbf': {'r': [], 'r_ot': []},
 'RF': {'r': [], 'r_ot': []}}